In [69]:
import torch
from transformers import BertTokenizer, BertModel
import numpy as np

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

2022-09-19 14:52:15.486708: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [83]:
sentences_list = []

file1 = open('house_mouse.txt', 'r')
sentences1 = file1.readlines()
  
for sentence in sentences1:
    sentences_list.append(sentence.strip())

file1 = open('computer_mouse.txt', 'r')
sentences2 = file1.readlines()

for sentence in sentences2:
    sentences_list.append(sentence.strip())

In [84]:
tokenized_sentences = []

for sentence in sentences_list:
    marked_text = "[CLS] " + sentence + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    tokenized_sentences.append(tokenized_text)

In [87]:
embeddings_list = []
mouse_indexes = []

model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True)
model.eval()

for tokenized_sentence in tokenized_sentences:

    mouse_indexes.append(tokenized_sentence.index("mouse"))

    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_sentence)
    segments_ids = [1] * len(tokenized_sentence)

    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        hidden_states = outputs[2]

        '''
        hidden_states has 4 dimensions:
        1. The layer number (13 layers)
        2. The batch number (sentences) [deleted]
        3. The word / token number (tokens in sentence)
        4. The hidden unit / feature number (768 features) 
        '''

    token_embeddings = torch.stack(hidden_states, dim=0)
    token_embeddings = torch.squeeze(token_embeddings, dim=1)

    embeddings_list.append(token_embeddings)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [93]:
for mouse_index in mouse_indexes:
    print(embeddings_list[0].numpy()[12][mouse_index][:4])

[-0.953875    0.24298051 -1.0363265  -0.5149844 ]
[-1.1949903  -0.01823178 -0.6367848  -0.38028663]
[-0.953875    0.24298051 -1.0363265  -0.5149844 ]
[-0.21474059 -0.3062644  -0.3999245  -0.30047092]
[ 0.57113004  0.12427469 -0.4557025  -0.39821926]
[-0.21474059 -0.3062644  -0.3999245  -0.30047092]
[-0.89742833  0.8151068  -0.5790246  -0.15906268]
[-0.21474059 -0.3062644  -0.3999245  -0.30047092]
[ 0.10403109 -0.10193337 -0.12502177 -0.28862375]
[ 0.10403109 -0.10193337 -0.12502177 -0.28862375]
[ 0.39346227 -0.13801533 -0.0295036  -0.79733586]
[-1.1949903  -0.01823178 -0.6367848  -0.38028663]
[-0.16911817  0.9123539  -0.7249186  -0.40225494]
[-0.4198347   1.1695786  -0.95079416 -0.48941252]
[ 0.27180743  0.9649347  -0.55659103 -0.90386796]
[-0.78540903  0.13333738 -0.9690666   0.14383869]
[-0.78540903  0.13333738 -0.9690666   0.14383869]
[-0.96983165  0.17588425 -0.17917037 -0.39939538]
[ 0.10403109 -0.10193337 -0.12502177 -0.28862375]
[-0.953875    0.24298051 -1.0363265  -0.5149844 ]


IndexError: index 41 is out of bounds for axis 0 with size 36